# Extraccion de datos de la API ALPHA VANTAGE

In [1]:
#Importacion de librerias
 
import requests
from pprint import pprint
import psycopg2
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
import pandas
from utils_apis import * 
import pandas as pd
from deltalake import DeltaTable
import pyarrow as pa
import os
from dotenv import load_dotenv




In [2]:
base_url = 'https://financialmodelingprep.com/api/v3'


In [27]:
# Creamos la session de spark para luego pasar como parametro

spark = SparkSession.builder \
    .appName("Guardar Delta Local") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .getOrCreate()


Conectarme a la base de datos donde tengo alojado la fecha de ultima extraccion

In [3]:
# OBTINE LA ULTIMA FECHA DE EXTRACCION
ultimaExtraccion= conexion_consultas(
    'pipeline.conf', 'postgresql','postgresql+psycopg2'
    )
# OBTENER LA SESSION
session= devolver_session( 'pipeline.conf', 'postgresql','postgresql+psycopg2')

print(ultimaExtraccion)
print(session)


2024-11-15 22:15:00


EXTRACCION INCREMENTAL STATELESS 

se extrae de: API de criptomonedas intradía(Proporciona datos de precios intradiarios de todas las criptomonedas que se comercializan en bolsas de todo el mundo.)


In [ ]:
load_dotenv(dotenv_path='.env')
api_key = os.getenv("API_KEY")


hiskvV2OvHYzOPvYgFhgb0JqXfk67SHy
environ({'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\Usuario\\AppData\\Roaming', 'APY_KEY': 'hiskvV2OvHYzOPvYgFhgb0JqXfk67SHy', 'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_11156_GLXMZOZHLEOVREIZ', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPUTERNAME': 'DESKTOP-CGL8DG7', 'COMSPEC': 'C:\\Windows\\system32\\cmd.exe', 'ELECTRON_RUN_AS_NODE': '1', 'FPS_BROWSER_APP_PROFILE_STRING': 'Internet Explorer', 'FPS_BROWSER_USER_PROFILE_STRING': 'Default', 'HADOOP_HOME': 'C:\\Program Files\\hadoop', 'HOMEDRIVE': 'C:', 'HOMEPATH': '\\Users\\Usuario', 'JAVA_HOME': 'C:\\Program Files\\Java\\jdk-11', 'JPY_INTERRUPT_EVENT': '1380', 'LOCALAPPDATA': 'C:\\Users\\Usuario\\AppData\\Local', 'LOGONSERVER': '\\\\DESKTOP-CGL8DG7', 'NUMBER_OF_PROCESSORS': '4', 'ONEDRIVE': 'C:\\Users\\Usuario\\OneDrive', 'ORIGINAL_XDG_

In [ ]:
""" 
Bloque donde se hace se arma la URL para hacer una peticion GET a la API,
y luego se pasa los parametros a la funcion get_data para convertir los datos obtenidos
en un formato JSON
"""
from datetime import datetime, timezone,timedelta
"""
se convierte la ultima fecha de extraccion a una zona horaria para poder restar con la fecha actual(la actual estaba adelantada)
y lograr traer desde la api solo las fechas posteriores a la ultima extraccion.
"""
fecha_extraccion = datetime.now(timezone.utc) - timedelta(hours=5, minutes=5)


endpoint1='historical-chart'
timeframe = "5min"
symbol = "ALIENUSD"
#symbol = "BTCUSD"
#from_date= date_now - relativedelta(months=1)

params = {
    "from": ultimaExtraccion ,
    "to": fecha_extraccion, 
    "apikey": api_key
  
}
endpoint = f"{endpoint1}/{timeframe}/{symbol}"
data = get_data(base_url, endpoint, params=params)

if data:
    # Obtener la fecha máxima de los datos recibidos
    fechas_datos = [(d['date']) for d in data if 'date' in d]
    if fechas_datos:
        fecha_maxima_datos = max(fechas_datos)
        print("Fecha máxima:", fecha_maxima_datos)

        # Actualizar la fecha de extracción con la fecha máxima obtenida
        actualizar_fecha_y_proceso(session, fecha_maxima_datos, 'extraccion incremental')
    else:
        print("No se encontraron datos en la respuesta de la API.")
else:
    print("No se obtuvieron datos de la API.")


print(data)
print(api_key)

Fecha máxima: 2024-11-16 10:10:00
Fecha y proceso actualizados correctamente.
[{'date': '2024-11-16 10:10:00', 'open': 0.00381017, 'low': 0.00381017, 'high': 0.00381017, 'close': 0.00381017, 'volume': 0.02361111111111111}, {'date': '2024-11-16 10:05:00', 'open': 0.00381017, 'low': 0.00381017, 'high': 0.00381017, 'close': 0.00381017, 'volume': 0.02361111111111111}, {'date': '2024-11-16 10:00:00', 'open': 0.00381017, 'low': 0.00381017, 'high': 0.00381017, 'close': 0.00381017, 'volume': 0.02361111111111111}, {'date': '2024-11-16 09:50:00', 'open': 0.00381017, 'low': 0.00381017, 'high': 0.00381017, 'close': 0.00381017, 'volume': 0.04722222222222222}, {'date': '2024-11-16 09:45:00', 'open': 0.00381017, 'low': 0.00381017, 'high': 0.00381017, 'close': 0.00381017, 'volume': 0.02361111111111111}, {'date': '2024-11-16 09:40:00', 'open': 0.00381017, 'low': 0.00381017, 'high': 0.00381017, 'close': 0.00381017, 'volume': 0.02361111111111111}, {'date': '2024-11-16 09:35:00', 'open': 0.00381017, 'low'

In [30]:
"""
Llamo a la funcion build_table para convertir el JSON en un DataFrame,
 y coloco un limite de salida para mostrar solo 20 registros del DF
"""
df1= build_table(data)
pd.set_option('display.max_rows', None) # sirve para mostrar todas las filas completas sin que acortar con ...

# agrego una columna al df para agregar el valor de la cripto
df1['symbol']= symbol
df1.head(20)

,date,open,low,high,close,volume,symbol
0,2024-11-15 22:15:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
1,2024-11-15 22:10:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
2,2024-11-15 22:05:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
3,2024-11-15 22:00:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
4,2024-11-15 21:50:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD
5,2024-11-15 21:45:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
6,2024-11-15 21:35:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD
7,2024-11-15 21:25:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD
8,2024-11-15 21:15:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD
9,2024-11-15 21:05:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD


In [31]:
# convierto la columna symbol a un string que pandas pueda intrepretar como string, porque mas arriba ya lo pase a
#  string pero pandas no reconoce explicitamente entre object y string
df1['symbol'] = df1['symbol'].astype('string')
df1.head(10)


,date,open,low,high,close,volume,symbol
0,2024-11-15 22:15:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
1,2024-11-15 22:10:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
2,2024-11-15 22:05:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
3,2024-11-15 22:00:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
4,2024-11-15 21:50:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD
5,2024-11-15 21:45:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
6,2024-11-15 21:35:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD
7,2024-11-15 21:25:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD
8,2024-11-15 21:15:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD
9,2024-11-15 21:05:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD


In [32]:
# consultar la cantidad de filas
print(f'la cantidad de fila es: {df1.shape[0]}')

la cantidad de fila es: 2981


Ingesta incremental

In [33]:
path= './mi_delta_table3'

result= guardar_en_tabla_delta(df1,
                        path,
                        partition_cols=None)
result

In [34]:
dtable = DeltaTable(path)
d=dtable.to_pandas()
d.head(10)

,date,open,low,high,close,volume,symbol
0,2024-11-15 19:50:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
1,2024-11-15 19:45:00,0.00373,0.00373,0.00373,0.00373,0.168056,ALIENUSD
2,2024-11-15 19:40:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
3,2024-11-15 19:35:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
4,2024-11-15 19:30:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
5,2024-11-15 19:25:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
6,2024-11-15 19:20:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
7,2024-11-15 19:15:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
8,2024-11-15 19:10:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
9,2024-11-15 19:05:00,0.00373,0.00373,0.00373,0.00373,0.084028,ALIENUSD
